In [1]:
from collections import defaultdict
import pathlib
import os
import sys
import argparse

import numpy as np
# from sklearn.linear_model import LinearRegression
# import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# from evaluate_pile_losses import evaluate_pile_losses
# from evaluate_pile_induction_criterias import evaluate_pile_induction_criterias

# import scipy.linalg
import torch
import torch.nn.functional as F
# import sklearn.cluster

import datasets
from datasets import load_dataset
datasets.enable_caching()
from transformers import AutoTokenizer, GPTNeoXForCausalLM

In [2]:
# ----- parse command line arguments -----
model_name = "pythia-70m"
step = 143000
# device = torch.device(args.device)
cache_dir = "/om/user/ericjm/quanta-discovery/cache/"
pile_cache_dir = "/om/user/ericjm/the_pile"
# loss_threshold = args.loss_threshold
# skip = args.skip
# num_tokens = args.num_tokens
# block_len = args.block_len
# output_dir = args.output_dir
num_load_documents = 200000
# filter_induction = args.filter_induction
# verbose = args.verbose

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    f"EleutherAI/{model_name}",
    revision=f"step{step}",
    cache_dir=os.path.join(cache_dir, model_name, f"step{step}"),
)

# ----- load the_pile test set -----
dataset = load_dataset("json", data_files=os.path.join(pile_cache_dir, "test.jsonl.zst"),
                        cache_dir=pile_cache_dir, split=f"train[:{num_load_documents}]") 

def tokenize_sample(sample):
    tokens = tokenizer(sample["text"], return_tensors='pt', 
                        max_length=1024, truncation=True)["input_ids"]
    return {"input_ids": tokens}

dataset = dataset.map(tokenize_sample, load_from_cache_file=True)
dataset = dataset.map(lambda sample: {"split_by_token": tokenizer.batch_decode(sample["input_ids"][0])}, load_from_cache_file=True)
dataset = dataset.map(lambda sample: {"tokens_len": len(sample["input_ids"][0])}, load_from_cache_file=True)
dataset = dataset.map(lambda sample: {"preds_len": max(sample["tokens_len"] - 1, 0)}, load_from_cache_file=True) # fixed this on 2023-02-06 to accomodate empty documents
starting_indexes = np.array([0] + list(np.cumsum(dataset["preds_len"])))

# save dataset
dataset.save_to_disk(os.path.join(pile_cache_dir, "the_pile_test_canonical_200k"))


Using custom data configuration default-8c41628c92058181
Found cached dataset json (/om/user/ericjm/the_pile/json/default-8c41628c92058181/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/200000 [00:00<?, ?ex/s]

  0%|          | 0/200000 [00:00<?, ?ex/s]

  0%|          | 0/200000 [00:00<?, ?ex/s]

  0%|          | 0/200000 [00:00<?, ?ex/s]

Saving the dataset (0/6 shards):   0%|          | 0/200000 [00:00<?, ? examples/s]

#### load it up as a test

In [5]:
dataset = datasets.load_from_disk(os.path.join(pile_cache_dir, "the_pile_test_canonical_200k"))

/om2/user/ericjm/miniconda3/envs/phase-changes/lib/python3.8/site-packages/datasets/arrow_dataset.py:1533: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [8]:
print(dataset[0]['split_by_token'])

['Roman', ' Catholic', ' Di', 'ocese', ' of', ' T', 'amb', 'ac', 'ound', 'a', '\n', '\n', 'The', ' Roman', ' Catholic', ' Di', 'ocese', ' of', ' T', 'amb', 'ac', 'ound', 'a', ' ()', ' is', ' a', ' di', 'ocese', ' located', ' in', ' the', ' city', ' of', ' T', 'amb', 'ac', 'ound', 'a', ' in', ' the', ' Ec', 'clesiast', 'ical', ' province', ' of', ' Dak', 'ar', ' in', ' Sen', 'egal', '.', '\n', '\n', 'History', '\n', ' August', ' 13', ',', ' 1970', ':', ' Establ', 'ished', ' as', ' Apost', 'olic', ' Pre', 'fecture', ' of', ' T', 'amb', 'ac', 'ound', 'a', ' from', ' the', ' Di', 'ocese', ' of', ' Ka', 'ol', 'ack', ' and', ' Di', 'ocese', ' of', ' Saint', '-', 'Louis', ' du', ' S', 'éné', 'gal', '\n', ' April', ' 17', ',', ' 1989', ':', ' Prom', 'oted', ' as', ' Di', 'ocese', ' of', ' T', 'amb', 'ac', 'ound', 'a', '\n', '\n', 'Special', ' churches', '\n', ' The', ' cathedral', ' is', ' Cath', 'é', 'dr', 'ale', ' Marie', ' Re', 'ine', ' de', ' l', '’', 'Univers', ' in', ' T', 'amb', 'ac', '

In [11]:
len(dataset[0]['split_by_token'])

306

In [13]:
for i in range(len(dataset)):
    if len(dataset[i]['split_by_token']) >= 1024:
        print(i)
        break

12


In [20]:
print(dataset[12]['preds_len'])
print(dataset[12]['tokens_len'])

1023
1024


In [25]:
len(tokenizer.batch_decode(tokenizer.encode(dataset[12]['text'])))

5071

In [26]:
tokenizer(dataset[12]['text'])

{'input_ids': [1532, 187, 15834, 27, 686, 510, 2554, 273, 253, 8820, 2605, 273, 247, 33217, 2685, 275, 22474, 8091, 15708, 4142, 275, 39174, 310, 271, 1527, 1953, 15, 844, 921, 285, 22048, 13, 347, 247, 1159, 273, 8091, 41299, 13, 253, 5921, 875, 253, 1554, 2865, 1079, 5289, 273, 32125, 285, 8091, 22959, 27, 19967, 1699, 18192, 8162, 18705, 281, 5233, 1481, 14, 251, 22959, 2429, 281, 362, 35149, 4811, 15, 844, 671, 9413, 253, 6349, 273, 25584, 14, 49292, 28384, 14, 1811, 84, 13, 534, 4566, 4457, 5697, 273, 41637, 4719, 285, 778, 5513, 253, 5233, 3116, 273, 29111, 275, 3626, 4870, 13, 824, 347, 253, 17000, 273, 9313, 275, 5890, 16173, 2464, 187, 7582, 27, 187, 14, 686, 41237, 416, 15, 20424, 15916, 8, 187, 14, 45802, 1240, 334, 3419, 7110, 394, 5973, 187, 14, 416, 2902, 21806, 527, 274, 48964, 187, 14, 5769, 83, 2016, 5801, 322, 1164, 274, 10734, 187, 34424, 27, 187, 14, 686, 709, 2582, 85, 4063, 2582, 2003, 15, 2383, 8, 187, 5564, 27, 686, 19013, 3836, 275, 308, 4063, 17827, 27, 19540,

In [27]:
dataset[12]['input_ids']

[[1532,
  187,
  15834,
  27,
  686,
  510,
  2554,
  273,
  253,
  8820,
  2605,
  273,
  247,
  33217,
  2685,
  275,
  22474,
  8091,
  15708,
  4142,
  275,
  39174,
  310,
  271,
  1527,
  1953,
  15,
  844,
  921,
  285,
  22048,
  13,
  347,
  247,
  1159,
  273,
  8091,
  41299,
  13,
  253,
  5921,
  875,
  253,
  1554,
  2865,
  1079,
  5289,
  273,
  32125,
  285,
  8091,
  22959,
  27,
  19967,
  1699,
  18192,
  8162,
  18705,
  281,
  5233,
  1481,
  14,
  251,
  22959,
  2429,
  281,
  362,
  35149,
  4811,
  15,
  844,
  671,
  9413,
  253,
  6349,
  273,
  25584,
  14,
  49292,
  28384,
  14,
  1811,
  84,
  13,
  534,
  4566,
  4457,
  5697,
  273,
  41637,
  4719,
  285,
  778,
  5513,
  253,
  5233,
  3116,
  273,
  29111,
  275,
  3626,
  4870,
  13,
  824,
  347,
  253,
  17000,
  273,
  9313,
  275,
  5890,
  16173,
  2464,
  187,
  7582,
  27,
  187,
  14,
  686,
  41237,
  416,
  15,
  20424,
  15916,
  8,
  187,
  14,
  45802,
  1240,
  334,
  3419,
  7110,
  